In [ ]:
from mp_api.client import MPRester

from solid_state.thermodynamics import (energy_above_hull,
                                        electrochemical_stability_window,
                                        chemical_mixing_energy,
                                        )

In [ ]:
def select_GGA_entry(entries):
    for entry in entries:
        if entry.parameters['run_type'] in ['GGA', 'GGA+U']:
            return entry

In [ ]:
API_KEY = 'your_api_key'

In [ ]:
with MPRester(API_KEY) as mpr:
    
    anode_coating = mpr.get_entry_by_material_id(['mp-13944'])      # Li3AlN2
    anode_coating = select_GGA_entry(anode_coating)

    electrolyte = mpr.get_entry_by_material_id(['mp-942733'])      # Li7La3Zr2O12
    electrolyte = select_GGA_entry(electrolyte)

    cathode_coating = mpr.get_entry_by_material_id(['mp-22961'])   # Li2ZnCl4
    cathode_coating = select_GGA_entry(cathode_coating)
    
    cathode = mpr.get_entry_by_material_id(['mp-552024'])          # Li0.5CoO2
    cathode = select_GGA_entry(cathode)

#### Electrolyte

In [ ]:
with MPRester(API_KEY) as mpr:
    entries_hull = mpr.get_entries_in_chemsys(electrolyte.composition.chemical_system, 
                                             additional_criteria={'thermo_types': ['GGA_GGA+U']})

In [ ]:
energy_above_hull(electrolyte, entries_hull)

In [ ]:
electrochemical_stability_window(electrolyte, entries_hull, ref_element='Li', return_phase_diagram=False)

#### Anode coating

In [ ]:
with MPRester(API_KEY) as mpr:
    entries_hull = mpr.get_entries_in_chemsys(anode_coating.composition.chemical_system, 
                                             additional_criteria={'thermo_types': ['GGA_GGA+U']})

In [ ]:
energy_above_hull(anode_coating, entries_hull)

In [ ]:
electrochemical_stability_window(anode_coating, entries_hull, ref_element='Li', return_phase_diagram=False)

In [ ]:
with MPRester(API_KEY) as mpr:
    chemsys_set = anode_coating.composition.chemical_system_set | electrolyte.composition.chemical_system_set
    entries_interface = mpr.get_entries_in_chemsys(chemsys_set, 
                                             additional_criteria={'thermo_types': ['GGA_GGA+U']})

In [ ]:
chemical_mixing_energy(anode_coating, electrolyte, entries_interface, return_phase_diagram=False)

#### Cathode coating

In [ ]:
with MPRester(API_KEY) as mpr:
    entries_hull = mpr.get_entries_in_chemsys(cathode_coating.composition.chemical_system, 
                                             additional_criteria={'thermo_types': ['GGA_GGA+U']},
                                             compatible_only=True)

In [ ]:
energy_above_hull(cathode_coating, entries_hull)

In [ ]:
electrochemical_stability_window(cathode_coating, entries_hull, ref_element='Li', return_phase_diagram=False)

In [ ]:
with MPRester(API_KEY) as mpr:
    chemsys_set = cathode_coating.composition.chemical_system_set | electrolyte.composition.chemical_system_set
    entries_interface = mpr.get_entries_in_chemsys(chemsys_set, 
                                             additional_criteria={'thermo_types': ['GGA_GGA+U']},
                                             )

In [ ]:
# electrolyte | coating interface
chemical_mixing_energy(cathode_coating, electrolyte, entries_interface, return_phase_diagram=False)

In [ ]:
with MPRester(API_KEY) as mpr:
    chemsys_set = cathode_coating.composition.chemical_system_set | cathode.composition.chemical_system_set
    entries_interface = mpr.get_entries_in_chemsys(chemsys_set, 
                                             additional_criteria={'thermo_types': ['GGA_GGA+U']})

In [ ]:
# coating | cathode interface
chemical_mixing_energy(cathode_coating, cathode, entries_interface,
                       return_phase_diagram=False,
                       open_element='Li',
                       relative_mu=-4.0, # charged state 
                       )